<a href="https://colab.research.google.com/github/meanjoo/auto_scoring/blob/master/%EC%98%81%EB%8B%A8%EC%96%B4_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#데이터 불러오기

In [ ]:
train_set = torchvision.datasets.EMNIST(
    root = './data/EMNIST',
    split = 'letters',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)
test_set = torchvision.datasets.EMNIST(
    root = './data/EMNIST',
    split = 'letters',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100)

#CNN 신경망 정의

In [ ]:
import torch
from torch import nn

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=100, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(in_features=100*7*7, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=26)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
learning_rate = 0.001
num_classes = 26
epochs = 10

- Cross-Entropy 손실 함수 : 신경망을 훈련하는 데 사용되는 가장 일반적인 손실 함수이다.
- Adam optimizer : 학습의 방향과 크기(Learning rate)를 모두 개선한 기법으로, 각 파라미터마다 다른 크기의 업데이트를 적용시킨다.

In [ ]:
import torch.optim as optim

In [ ]:
net = NeuralNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#학습

In [ ]:
import pandas as pd
from collections import OrderedDict as od
from IPython.display import clear_output

In [ ]:
pd_results = []

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        out = net(images)
        loss = criterion(out, labels-1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total = labels.size(0)
        preds = torch.max(out.data, 1)[1]
        correct = (preds==labels-1).sum().item()

        if (i+1)%200==0:
            results = od()
            results['epoch'] = epoch+1
            results['idx'] = i+1
            results['loss'] = loss.item()
            results['accuracy'] = 100.*correct/total
            pd_results.append(results)
            df = pd.DataFrame.from_dict(pd_results, orient='columns')

            clear_output(wait=True)
            display(df)

,epoch,idx,loss,accuracy
0,1,200,0.508297,84.0
1,1,400,0.550647,85.0
2,1,600,0.315971,88.0
3,1,800,0.414611,85.0
4,1,1000,0.700541,78.0
...,...,...,...,...
85,15,400,0.316905,90.0
86,15,600,0.087509,96.0
87,15,800,0.290601,91.0
88,15,1000,0.425378,85.0


#테스트

In [ ]:
net.eval()

correct, total = 0, 0
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        out = net(images)
        preds = torch.max(out.data, 1)[1]
        correct += (preds==labels-1).sum().item()
        total += len(labels)

    print("Test accuracy: ", 100.*correct/total)

Test accuracy:  93.20192307692308
